In [23]:
%run utils_rf.py

In [24]:
#read training data
file = "../tables/model_input/noc_answers.csv"
x, x_agg, y, y_agg, x_noclvl, y_noclvl = data_proccess(file,True)

#grab just the noc codes to cut out test nocs from main noc table
train_nocs = pd.read_csv(file,usecols=['noc_code']).drop_duplicates()

In [25]:
#read full noc set
file ="../tables/model_input/noc_scores.csv"
nocs = pd.read_csv(file,index_col=['noc'])
nocs.sort_index(inplace=True)
nocs.drop(['Unnamed: 0'],axis=1,inplace=True)
nocs = np.round(nocs).astype(int)
nocs.drop(train_nocs['noc_code'],inplace=True)

In [26]:
f = open('../tables/SFFS_output/inc_features.txt', 'r')
sffs_prob_features = f.readlines()
f.close()

sffs_prob_features = [x[:-1] for x in sffs_prob_features]

Train on whole training set (see testing files for k_fold tests)

In [27]:
nocs_inc_features = nocs.loc[:,sffs_prob_features].copy()

rf = RandomForestClassifier(**init_params('cat'))
rf.fit(x.loc[:,sffs_prob_features],y['increase'])
noc_preds = pd.Series(rf.predict_proba(nocs_inc_features)[:,0],index=nocs.index)

In [28]:
noc_preds.to_csv(r'../tables/model_output/inc_projections.csv'
                 ,index=True,header=True)

Now for the decrease model

In [29]:
f = open('../tables/SFFS_output/dec_features.txt', 'r')
sffs_prob_features = f.readlines()
f.close()

sffs_prob_features = [x[:-1] for x in sffs_prob_features]

In [30]:
nocs_dec_features = nocs.loc[:,sffs_prob_features].copy()

rf = RandomForestClassifier(**init_params('cat'))
rf.fit(x.loc[:,sffs_prob_features],y['decrease'])
noc_preds = pd.Series(rf.predict_proba(nocs_dec_features)[:,0],index=nocs.index)

In [31]:
noc_preds.to_csv(r'../tables/model_output/dec_projections.csv'
                 ,index=True,header=True)